In [1]:
#!pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [2]:
import numpy as np
import re
import twint
import datetime as dt 
import os 
import pandas as pd
import nest_asyncio
import glob
import shutil
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nest_asyncio.apply() # for working in jupyter notebooks 

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopwords = stopwords.words('english')
stopwords = set(stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sophielogan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sophielogan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
frame = pd.read_csv('/users/sophielogan/Desktop/tweets_downloads/20200406chicago.csv')

In [4]:
# Let's keep it in 45 words. We can loop every vocabulary
potential_crime_string = '''shooting killed kill shot gun rifle pistol murder murdered murderers violence violent attack mug mugged mugging assault assaulted assaulting criminal criminals harass harassment sexual rape rapist raped
raping hit punch punched kick stab weapon knife wound hurt injured wounded offense'''
''' felony misdemeanor law abuse force''' # battered'''
#potential_crime_string = '''gang band danger dangerous kidnap kidnapped kidnapping robber robbery steal stole stolen hijacking victim
#victims trafficking bomb police fbi death dead died die
#'''
pcs_split = potential_crime_string.split()
print(len(pcs_split))
pcs_split_or = " OR ".join(pcs_split)
pcs_split_or

40


'shooting OR killed OR kill OR shot OR gun OR rifle OR pistol OR murder OR murdered OR murderers OR violence OR violent OR attack OR mug OR mugged OR mugging OR assault OR assaulted OR assaulting OR criminal OR criminals OR harass OR harassment OR sexual OR rape OR rapist OR raped OR raping OR hit OR punch OR punched OR kick OR stab OR weapon OR knife OR wound OR hurt OR injured OR wounded OR offense'

In [5]:
social = '''sociable, gregarious societal friendly society socialization political  sociality 
                        interpersonal  ethnic socially party welfare public community socialist societies development
                            network humans socialism collective personal corporation social constructivism
                        relations volition citizenship brute   attitude rights socio 
                        socioeconomic ethics civic communal marital  sociale socialized communities     
                         policy   unions        
                        institutions values     governmental   organizations jamboree 
                         festivity    fairness  support  care  
                         sides   activism     unsocial psychosocial 
                        socializing psychological distributional  demographic  participation reunion 
                        partygoer partyism festive power network gala housewarming celebration counterparty   social-war
                        particularist interactional ideational asocial'''

In [6]:
health = '''disease obesity world health organization medicine nutrition well-being exercise welfare wellness health care public health 
                     nursing stress safety hygiene research social healthy condition aids epidemiology healthiness wellbeing
                     care illness medical dieteducation infectious disease environmental healthcare physical fitness hospitals 
                     health care provider doctors healthy community design insurance sanitation human body patient mental health
                      medicare agriculture health science fitnesshealth policy  weight loss physical therapy psychology pharmacy
                     metabolic organism human lifestyle status unhealthy upbeat vaccination sleep condom alcohol smoking water family
                     eudaimonia eudaemonia air house prevention genetics public families poor needs treatment communicable disease 
                     study protection malaria development food priority management healthful mental provide department administration
                     programs help assistance funding environment improving emergency need program affected schools private mental illness 
                     treat diseases preparedness perinatal fertility sickness veterinary sanitary pharmacists behavioral midwives
                     gerontology infertility hospitalization midwifery cholesterol childcare pediatrician pediatrics medicaid asthma 
                     pensions sicknesses push-up physical education body-mass-index eat well gymnastic apparatus tune up good morning 
                     bathing low blood-pressure heart attack health club ride-bike you feel good eczema urticaria dermatitis sunburn overwork 
                     manufacturing medical sociology need exercise run'''

In [7]:
'''Vectorizing the sets of words, then standardizing them. TFIDF will be used in order to take care of the least 
frequent words. Standardizing is cause TFIDF favors long sentences and there'll be inconsistencies between the length 
of the tweets and the length of set of words.'''

def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = TfidfVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()


In [8]:
socialvector = get_vectors(social)
healthvector = get_vectors(health)

/Users/sophielogan/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass input=['sociable, gregarious societal friendly society socialization political  sociality \n                        interpersonal  ethnic socially party welfare public community socialist societies development\n                            network humans socialism collective personal corporation social constructivism\n                        relations volition citizenship brute   attitude rights socio \n                        socioeconomic ethics civic communal marital  sociale socialized communities     \n                         policy   unions        \n                        institutions values     governmental   organizations jamboree \n                         festivity    fairness  support  care  \n                         sides   activism     unsocial psychosocial \n                        socializing psychological distributional  demographic  participation reunion \

In [9]:
def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp

In [10]:
def clean_all_tweets(frame):
    frame['tweet'] = frame['tweet'].map(lambda x: clean_tweet(x))
    
clean_all_tweets(frame)

<ipython-input-9-a6606d7cc17b>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(tweet) == np.float:


In [11]:
frame.tweet

0     yes even kill nobody gone see em win win regar...
1     one thing never understood purge everyone want...
2                              kill local indie softboy
3                        government issued getting shot
4                      1 good gun never worry money smh
                            ...                        
95                                     ooh one hit home
96    never drinking 4 loko ever life shit almost ki...
97                                            hit lmfao
98    attack time full creativity keep innovative mi...
99    part dealing pandemic one thing major thing co...
Name: tweet, Length: 100, dtype: object

In [13]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

def sentiment(df): 
    sid = SentimentIntensityAnalyzer() # Calculate neutral, negative and positive sentiment scores and label tweet with highest sentiment 
    df['sentiment'] = np.nan
    for idx, tweet in enumerate(df['tweet']):
        sentiment_dict = sid.polarity_scores(tweet)
        sentiment_names = ('Negative', 'Positive', 'Neutral')
        sentiment_tup = (sentiment_dict['neg'], sentiment_dict['pos'], sentiment_dict['neu'])
        if np.any(sentiment_tup) > 0.0: # if all values are 0, leave label as nan 
            index = sentiment_tup.index(max(sentiment_tup))
            df['sentiment'][idx] = sentiment_names[index]  
            
sentiment(frame)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sophielogan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
<ipython-input-13-d3918884e414>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'][idx] = sentiment_names[index]
/Users/sophielogan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [23]:
# Mentions any items in dictionary, count 1 
def naive_count(category, tweet): 
    for word in tweet: 
        if word in category:
            return 1 
        return 0 

frame['naive count social'] = frame['tweet'].map(lambda x: naive_count(social, x))
frame['naive count health'] = frame['tweet'].map(lambda x: naive_count(health, x))

In [14]:
## Vectorizing the tweets
tv=TfidfVectorizer()
# tweets_bowl = tweets_bowl.tweets.apply(get_vectors)
# tweets_bowl.head()
tfidf_tweets =tv.fit_transform(frame.tweet)

In [15]:
'''Jaccard similarity is good for cases where duplication does not matter, 
cosine similarity is good for cases where duplication matters while analyzing text similarity. For two product descriptions, 
it will be better to use Jaccard similarity as repetition of a word does not reduce their similarity.'''

def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)
# jaccard_score(socialvector, economic_vector)

#for similarity of 1 and 2 of column1
# jaccard_similarity('dog lion a dog','dog is cat')
def get_scores(title, group, frame):
    frame[f'jaccard sim score {title}'] = frame['tweet'].map(lambda x: jaccard_similarity(group,x))

get_scores('social',social,frame)
get_scores('health',health,frame)

#def get_scores(group,tweets):
#    scores = []
#    for tweet in tweets:
#        s = jaccard_similarity(group, tweet)
#        scores.append(s)
#    return scores

In [28]:
frame.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,reply_to,retweet_date,translate,trans_src,trans_dest,sentiment,jaccard sim score social,jaccard sim score health,naive count social,naive count health
0,1247312570698084352,1247309890554560512,2020-04-06 18:57:45 CDT,2020-04-06,18:57:45,-600,3388071358,pamperedone,Tiffany,NaN,...,"[{'screen_name': 'kiss_myROSE', 'name': '👑 Sha...",NaN,NaN,NaN,NaN,Positive,0.607143,0.629630,1,1
1,1247312303218966528,1247312303218966528,2020-04-06 18:56:41 CDT,2020-04-06,18:56:41,-600,334571553,jae_khaliah,☀︎︎ JAÉ KHALIAH ☽,NaN,...,[],NaN,NaN,NaN,NaN,Neutral,0.750000,0.777778,1,1
2,1247311806806319115,1247300921354223619,2020-04-06 18:54:43 CDT,2020-04-06,18:54:43,-600,3301158974,miserabletop,jared ❤️‍🔥,NaN,...,"[{'screen_name': 'MarshallIversen', 'name': 'M...",NaN,NaN,NaN,NaN,Negative,0.535714,0.555556,1,1
3,1247310908172439554,1247310908172439554,2020-04-06 18:51:09 CDT,2020-04-06,18:51:09,-600,2784851391,ededdnedgar,Investor Ed 🏚🏢🏦💰📈📙,NaN,...,[],NaN,NaN,NaN,NaN,Neutral,0.500000,0.518519,1,1
4,1247309543157178369,1247307527592837120,2020-04-06 18:45:43 CDT,2020-04-06,18:45:43,-600,1028356298503532555,diegoo_dela,lover boy,NaN,...,"[{'screen_name': 'CeetTheKid', 'name': 'ceetot...",NaN,NaN,NaN,NaN,Positive,0.482759,0.500000,0,0


In [ ]:
# social scores
#s_scores1 = get_scores(social, frame.tweet.to_list())
#s_scores2 = get_scores(health, frame.tweet.to_list())
#print(s_scores1[:10])
#print(s_scores2[:10])

In [ ]:
#frame['murder score'] = s_scores1
#frame['rape score'] = s_scores2
#frame['robbery score'] = s_scores3
#frame['aggravated assault score'] = s_scores4

In [ ]:
#Create dictionary 
#'theft':['grand theft auto','putin is a murderer','murder raste'],'rape':

In [ ]:
# Cosine rather than Jaccard 
# Similarity 
# Use TF, not TFIDF -- TFIDF is for longer documents, not to small strings like twitter 
# Add counter as well to compare to TF 

# Very simple way to do topic modelling - rather than LDA 
# Put LDA topic modelling code and then say this helped guide us to do manual dictionary for topic models 

# Merge based on year 


# Visuals
# bar chart: x axis = city, y axis = avg norm measure, and then bars 1st bar actual measure (fbi), second bar sentiment, third bar normalized tweet count
# line graph see how each metric tracks over time
# map of cities and do it for metric seperateky
    

In [ ]:
# Later analysis 

# Normalization after merging (normalize by population, divide by max (max ratio in any city) to make it standardized) 

# Use frame.near to get city 
#tweets_agg = tweets_df.groupby(['city','year']).mean()

#fbi_df.merge(tweets_agg,
#            how = 'left',
#            left_on = ['city','year'],
#            right_on = ['city','year'])